In [16]:
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.automap import automap_base
import pandas as pd
from sqlalchemy.orm import Session

import matplotlib

In [24]:
engine = create_engine("sqlite:///PlayerStats.sqlite", echo=True)

In [25]:
Base = automap_base()
Base

sqlalchemy.ext.automap.Base

In [26]:
Base.prepare(engine, reflect=True)
Base

2021-02-06 14:02:26,481 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-06 14:02:26,506 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 14:02:26,529 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-06 14:02:26,537 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 14:02:26,567 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-02-06 14:02:26,584 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 14:02:26,590 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("data")
2021-02-06 14:02:26,601 INFO sqlalchemy.engine.base.Engine ()
2021-02-06 14:02:26,622 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-02-06 14:02:26,627 INFO sqlalchemy.engine.base.Engine ('data',)
2021-02-06 14:02:26,631 INFO sqlalchemy.engine

sqlalchemy.ext.automap.Base

In [32]:
Base.classes

In [3]:
inspector = inspect(engine)
inspector.get_table_names()

['data']

In [15]:
columns = inspector.get_columns('data')
for x in columns:
    print(x['name'], x['type'])

index BIGINT
Player TEXT
Pos TEXT
Age BIGINT
Tm TEXT
G BIGINT
GS BIGINT
MP BIGINT
FG BIGINT
FGA BIGINT
FG% FLOAT
3P BIGINT
3PA BIGINT
3P% FLOAT
2P BIGINT
2PA BIGINT
2P% FLOAT
eFG% FLOAT
FT BIGINT
FTA BIGINT
FT% FLOAT
ORB BIGINT
DRB BIGINT
TRB BIGINT
AST BIGINT
STL BIGINT
BLK BIGINT
TOV BIGINT
PF BIGINT
PTS BIGINT
Season BIGINT
Country TEXT


In [12]:
Base = automap_base()

In [14]:
Base.classes.keys()

[]

In [18]:
session = Session(engine)

In [28]:
mins_country = engine.execute().fetchall()

In [10]:
df = pd.read_sql("SELECT Player, MP, Country FROM data", con=engine)

In [11]:
df

,Player,MP,Country
0,Steven Adams,1680,New Zealand
1,Bam Adebayo,2417,United States
2,LaMarcus Aldridge,1754,United States
3,Kyle Alexander,13,Canada
4,Nickeil Alexander-Walker,591,Canada
...,...,...,...
13559,Haywoode Workman,2034,United States
13560,James Worthy,3008,United States
13561,Howard Wright,164,United States
13562,A.J. Wynder,39,United States
